In [1]:
import cv2
import math
import numpy as np
import matplotlib.pylab as plt
from glob import glob

In [3]:
images_list = glob(r"../data/*.png")
#image = cv2.imread(images_list[30])
image = cv2.imread(r"..\data\IMG_4109.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [4]:
# Copy edges to the images that will display the results in BGR
edged = cv2.Canny(blurred, 50, 200, None, 3)
image_with_lines = cv2.cvtColor(edged, cv2.COLOR_GRAY2BGR)

# Probabilistic Line Transform
#linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 20, 3, 2, 10)
linesP = cv2.HoughLinesP(edged, 1, np.pi / 180, 20, 3, 30, 10)
# Draw the lines
if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(image_with_lines, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)
print(len(linesP), "Lines were found.")

126 Lines were found.


In [5]:
# Apply Laplacian of Gaussian
blobs_log = cv2.Laplacian(image_with_lines, cv2.CV_64F)
blobs_log = np.uint8(np.absolute(blobs_log))
    
# Set up the detector with default parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 10
params.maxThreshold = 1000

# Filter by Area.
params.filterByArea = True
params.minArea = 100
params.maxArea = 1000

# Filter by Circularity
params.filterByCircularity = False
params.minCircularity = 0.3
params.maxCircularity = 0.9

# Filter by Convexity
params.filterByConvexity = False
params.minConvexity = 0.1
params.maxConvexity = 1

# Filter by Inertia
params.filterByInertia = False
params.minInertiaRatio = 0.01
params.maxInertiaRatio = 1

# Create a detector with the parameters
detector = cv2.SimpleBlobDetector_create(params)

# Detect blobs.
keypoints = detector.detect(blobs_log)
print(len(keypoints), "Keypoints were found.")
# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of the blob
im_with_keypoints = cv2.drawKeypoints(image_with_lines, keypoints, np.array([]), (0, 255, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

17 Keypoints were found.


In [6]:
# Show keypoints
winname1="Keypoints"
cv2.namedWindow(winname1) 
cv2.moveWindow(winname1, 500,-2000)
cv2.imshow(winname1, im_with_keypoints)

#cv2.imshow("cdstP", cdstP)
cv2.imshow("Edged", edged)

cv2.waitKey()
cv2.destroyAllWindows()